In [1]:
import cvxopt as cvx
import numpy as np
import numpy.matlib
import picos as pic
import control

diagBlock = cvx.matrix(np.identity(4))
zeroesBlock = cvx.matrix(numpy.matlib.zeros((4, 4)))

print('Initial conditions are:')
print()

A = cvx.matrix([[zeroesBlock, zeroesBlock], [diagBlock, zeroesBlock]])
print('A = ', A)

B = cvx.matrix([zeroesBlock, diagBlock])
print('B = ', B)

I8 = cvx.matrix(np.identity(8)) #identity matrix of size 8
I4 = diagBlock #identity matrix of size 4

beta = 1000    #guaranteed rate of convergence
mu = 1      #coefficient
print('beta =', beta, 'mu =', mu)

C = control.ctrb(A, B)

print('Controllability matrix rank is:', np.linalg.matrix_rank(C))

problem = pic.Problem()

A = pic.new_param('A', A)
B = pic.new_param('B', B)
I8 = pic.new_param('I8', I8)
I4 = pic.new_param('I4', I4)
beta = pic.new_param('beta', beta)
mu = pic.new_param('mu', mu)

P = problem.add_variable(name='P', size=(8, 8), vtype='symmetric')
Y = problem.add_variable(name='Y', size=(4, 8))

problem.add_constraint(((A*P + P*A.T + B*Y + Y.T*B.T & P) // (P & -I8/beta)) <= 0, 'first_constr')
problem.add_constraint(((A*P + P*A.T + mu*(B*Y + Y.T*B.T) & P) // (P & -I8/beta)) <= 0, 'second_constr')
problem.add_constraint(((P & Y.T) // (Y & I4/(mu*mu))) >= 0, 'third_constr')

problem.set_objective('max', 'I'|P)

print(problem)
problem.solve(verbose=0, solver='cvxopt')

print()
print('The optimal P is:')
print(P)
print('The optimal Y is:')
print(Y)

P = np.array(P.value)
Y = np.array(Y.value)

P_inv = np.linalg.inv(P)

print('Inverse P is:')
print(P_inv)

print('Optimal K is:')
print(np.dot(Y, P_inv))

Initial conditions are:

A =  [ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  1.00e+00  0.00e+00  0.00e+00 ... ]
[ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  1.00e+00  0.00e+00 ... ]
[ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  1.00e+00 ... ]
[ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00 ... ]
[ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00 ... ]
[ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00 ... ]
[ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00 ... ]
[ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00 ... ]

B =  [ 0.00e+00  0.00e+00  0.00e+00  0.00e+00]
[ 0.00e+00  0.00e+00  0.00e+00  0.00e+00]
[ 0.00e+00  0.00e+00  0.00e+00  0.00e+00]
[ 0.00e+00  0.00e+00  0.00e+00  0.00e+00]
[ 1.00e+00  0.00e+00  0.00e+00  0.00e+00]
[ 0.00e+00  1.00e+00  0.00e+00  0.00e+00]
[ 0.00e+00  0.00e+00  1.00e+00  0.00e+00]
[ 0.00e+00  0.00e+00  0.00e+00  1.00e+00]

beta = 1000